In [ ]:
import os
from xml.etree import ElementTree

tree = ElementTree.parse("../data/iam-docdb-1.0/851.inkml")

# Notes

So, it appears that an inkml document is structured as follows:
- A list of "trace" elements. Traces have their own special grammar for expressing a sequence of points, presumably because of how verbose XML is?
- A nexted set of textView nodes at the bottom. This expresses the actual structure of the document... there's a top-level "document" view, several nodes underneath that like "drawing" and "text block", and some other stuff. These nodes reference the trace elements by id.

I'm currently mostly interested in the text! One approach would be to:
- build an id->tracedata map from the traces
- scan through all nodes in the view tree... and if there's a transcription, append all the recursively-referenced traces into one big ink.

InkML is [complicated](https://www.w3.org/TR/InkML/), but thankfully the docs seem fairly regularly structured, so I should be able to grab the relevant stuff without a proper parser or anything.

In [ ]:
tree

In [ ]:
traces = tree.findall("./trace")

In [ ]:
import numpy as np

mapping = tree.find(".//mapping")
mapping.attrib['type']
matrix = mapping.find(".//matrix")
m = np.array([[float(f) for f in line.split(" ")] for line in matrix.text.split(",")[:-1]])[:3, :3]
m * m

In [ ]:
traces

In [ ]:
trace = traces[3]

In [ ]:
trace.text.split(",")

In [ ]:
import re
testlit = '1125 18432,\'23\'43,"7"-8,3-5,7 -3,6 2,6 8'
numbers = [[float(n) for n in re.findall(r'-?[0-9]*\.?[0-9]*', line) if n] for line in testlit.split(",")]

In [ ]:
import numpy as np

points = []
arrays = list(np.array(numbers))
position = arrays[0]
points.append(position)
velocity = arrays[1]
position = position + velocity
points.append(position)
for acceleration in arrays[2:]:
    velocity = velocity + acceleration
    position = position + velocity
    points.append(position)
    
points

In [ ]:
import re

def parse_trace_string(text):
    # NB: we assume the usual pattern of pos, velocity, accel, accel...
    # because it's really annoying to parse out the actual sigils.
    numbers = [
        [float(n) for n in re.findall(r'-?[0-9]*\.?[0-9]*', line) if n]
        for line in text.split(",")
    ]
    
    points = []
    arrays = list(np.array(numbers))
    position = arrays[0]
    points.append(position)
    velocity = arrays[1]
    position = position + velocity
    points.append(position)
    for acceleration in arrays[2:]:
        velocity = velocity + acceleration
        position = position + velocity
        points.append(position)
        
    array = np.array(points)
    array[:, 3] = 0.0
    array[1, 3] = 1.0
    array[-1, 3] = -1.0

    return array

In [ ]:
id_to_trace = {
    trace.attrib['{http://www.w3.org/XML/1998/namespace}id']: parse_trace_string(trace.text)
    for trace in traces
}

In [ ]:
tree.findall('.//traceView')

examples = []
for view in :
    annotation = view.find('./annotation')
    if annotation is None:
        continue
    node_type = annotation.text
    if not (node_type in ['Textline', 'Word']):
        continue
    transcription = view[1].text
    traces = [
        trace.attrib['traceDataRef'][1:]
        for trace
        in view.findall('.//traceView[@traceDataRef]')
    ]
    examples.append((transcription, np.concatenate([id_to_trace[t] for t in traces])))